# Evaluation

In [1]:
import itertools # 是 Python 的內建模組，提供了一組用於處理迭代器的函數和工具。
                 # 它包含了各種用於高效處理迭代器的函數，可以幫助我們編寫更簡潔、高效的代碼。
import sys # 是 Python 的內建模組，提供了與 Python 解釋器和運行環境相關的功能。

import numpy as np

# sys.path 是一個列表，包含了 Python 解釋器在導入模組時會搜尋的路徑。
# 當你使用 import 語句導入模組時 Python 會依次在 sys.path 中的路徑下尋找對應的模組文件。
sys.path.append("../ecg-classification/")
# sys.path.append("C:\\Users\\Chen_Lab01\\Documents\\GitHub/ecg-classification")
# from IPython.display import Video
# import cv2
from matplotlib import pyplot as plt
plt.style.use("ggplot") #  是 Matplotlib 庫中用於設置繪圖樣式的函數。它使用了一種名為 "ggplot" 的預定義樣式
import matplotlib
matplotlib.use("TkAgg")
                        #  該樣式模仿了 R 語言的 ggplot2 繪圖包的外觀。
# print(sys.path)
import torch
from ecg_tools.config import EcgConfig, Mode
from ecg_tools.data_loader import DatasetConfig, get_data_loaders
from ecg_tools.model import ECGformer
from ecg_tools.train import ECGClassifierTrainer


## Load model

In [14]:
import torch
config = EcgConfig()    
model_quantized = torch.load("..\\..\\model_save\\model_quantized_148.pth")
model = torch.load("..\\..\\model_save\\model_epoch_148.pth")
model_quantized.eval()
model_quantized.to('cpu')
model.eval()
model.to('cpu')

C:\Users\Xaio\anaconda3\envs\pytorch-ecg\lib\site-packages\torch\_utils.py:382: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  device=storage.device,


ECGformer(
  (encoder): ModuleList(
    (0): TransformerEncoderLayer(
      (0): ResidualAdd(
        (block): Sequential(
          (0): MultiHeadAttention(
            (queries_projection): Linear(in_features=16, out_features=16, bias=True)
            (values_projection): Linear(in_features=16, out_features=16, bias=True)
            (keys_projection): Linear(in_features=16, out_features=16, bias=True)
            (final_projection): Linear(in_features=16, out_features=16, bias=True)
          )
          (1): Dropout(p=0.1, inplace=False)
        )
      )
      (1): ResidualAdd(
        (block): Sequential(
          (0): MLP(
            (0): Linear(in_features=16, out_features=16, bias=True)
            (1): ReLU()
            (2): Linear(in_features=16, out_features=16, bias=True)
          )
          (1): Dropout(p=0.1, inplace=False)
        )
      )
    )
  )
  (classifier): Classifier(
    (0): Reduce('b n e -> b e', 'mean')
    (1): Linear(in_features=16, out_features=6,

## 量化模型

In [12]:
# import torch.quantization
# 
# # 使用 Eager Mode Quantization
# # 將 torch.nn.Linear 的參數映射到 -127~127 之間
# 
# model_quantized = torch.quantization.quantize_dynamic(
#     model, {torch.nn.Linear}, dtype=torch.qint8
# )
# 
# torch.save(model_quantized, "..\\..\\model_save\\model_quantized_148.pth")

## 準確度測試

In [52]:
import einops
loader = get_data_loaders(DatasetConfig())
accuracy = 0
for signal, label in loader[Mode.eval]:
    signal.to('cpu')
    label.to('cpu')
    signal = einops.rearrange(signal, "b c e -> b e c")
    # print(signal)
    p = model_quantized(signal)
    # print(p)
    print(label)
    # print(signal.shape, label.shape)
    # print(p.argmax(1) == label)
    accuracy += torch.sum(p.argmax(1) == label)
    print(f"accuracy: {accuracy / config.dataset.batch_size}")
    break

tensor([0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 5, 0, 5, 0, 0,
        2, 0, 4, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 4, 0, 5, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 4])
accuracy: 0.890625


## View Prameter

Model layer

In [53]:
print(model_quantized)

ECGformer(
  (encoder): ModuleList(
    (0): TransformerEncoderLayer(
      (0): ResidualAdd(
        (block): Sequential(
          (0): MultiHeadAttention(
            (queries_projection): DynamicQuantizedLinear(in_features=16, out_features=16, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (values_projection): DynamicQuantizedLinear(in_features=16, out_features=16, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (keys_projection): DynamicQuantizedLinear(in_features=16, out_features=16, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (final_projection): DynamicQuantizedLinear(in_features=16, out_features=16, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
          )
          (1): Dropout(p=0.1, inplace=False)
        )
      )
      (1): ResidualAdd(
        (block): Sequential(
          (0): MLP(
            (0): DynamicQuantizedLinear(in_features=16, out_features=16, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
       

字典形式的量化模型參數

In [ ]:
# 量化
for param_name, param_tensor in model_quantized.state_dict().items():
    print(f"{param_name}")

字典形式的非量化模型參數

In [ ]:
# 未量化
for param_name, param_tensor in model.state_dict().items():
    print(f"{param_name}")

## Packed_params

In [54]:
import numpy as np
weight_tensor_after = model_quantized.state_dict()['encoder.0.0.block.0.final_projection._packed_params._packed_params']

# packed_params = model_quantized.encoder[0][0].block[0].queries_projection._packed_params._packed_params

# Unpack the quantized weights and biases
int8_weights, int8_bias = torch.ops.quantized.linear_unpack(packed_params)
int8_weights_nd = np.array(int8_weights.int_repr())
int8_bias_nd = int8_bias.detach().numpy()
# # Dequantize the weights and biases
# weights = int8_weights.dequantize()
# bias = int8_bias.dequantize()

print(int8_weights_nd)#　將量化後的權重轉換為整數表示並轉化為numpy# array
print(int8_weights.q_scale()) #　獲取量化的scale
print(int8_bias_nd)

NameError: name 'packed_params' is not defined

In [ ]:
for param_name, param_tensor in model_quantized.state_dict().items():
    if isinstance(param_tensor, torch.Tensor):
        if not any(special_param in param_name for special_param in ['scale', 'zero_point', 'dtype', '_packed_params']):
            print(f"{param_name}\t{param_tensor.size()}")

印出模型中所有的weights和bias

In [ ]:
for name, param in model_quantized.named_parameters():
    if 'weight' in name:
        print(f"Layer: {name}")
        print(f"Weight: {param.data}")
    elif 'bias' in name:
        print(f"Layer: {name}") 
        print(f"Bias: {param.data}")


In [ ]:
for var_name in model_quantized.state_dict():
    print(var_name, "\t", model_quantized.state_dict()[var_name])

## Parameter Extraction

In [ ]:
import numpy as np

model_state_dict = model_quantized.state_dict()

ignore_keys = [
    'scale',
    'zero_point',
    '_packed_params.dtype'
]

with open('model_layers_params.txt', 'w') as f:
    for layer_name, param_tensor in model_state_dict.items():
        if not any(ignore_key in layer_name for ignore_key in ignore_keys):
            if '_packed_params._packed_params' in layer_name:
                # Dynamically get the corresponding layer
                layer_parts = layer_name.split('.')
                layer = model_quantized
                for part in layer_parts[:-1]:
                    layer = getattr(layer, part)
                
                packed_params = getattr(layer, '_packed_params')
                
                # Unpack the quantized weights and biases
                int8_weights, int8_bias = torch.ops.quantized.linear_unpack(packed_params)
                int8_weights_nd = np.array(int8_weights.int_repr())
                int8_bias_nd = int8_bias.detach().numpy()
                
                f.write(f"Layer: {layer_name}\n")
                f.write(f"Quantized Weights:\n{int8_weights_nd}\n")
                f.write(f"Quantization Scale: {int8_weights.q_scale()}\n")
                f.write(f"Quantized Bias:\n{int8_bias_nd}\n")
                f.write("\n")
            else:
                # Convert param_tensor to ndarray
                param_ndarray = param_tensor.detach().numpy()
                
                f.write(f"Layer: {layer_name}\n") 
                f.write(f"Parameters:\n{param_ndarray}\n")
                f.write("\n")


### 提取六層 Encoder, Classifier, Embedding 參數

### 32-bit 浮點數(Unquantized model)

In [ ]:
import os
import numpy as np
import torch

model_state_dict = model.state_dict()

def extract_params(layer_name, param_tensor, folder_path):
    param_ndarray = param_tensor.detach().numpy()
    
    # Save parameters as 32-bit float representations
    param_file = os.path.join(folder_path, 'parameters.txt')
    with open(param_file, 'w') as f:
        if len(param_ndarray.shape) == 2:
            for col in param_ndarray.T:
                for value in col:
                    f.write(f"{value:.8e}\n")
        else:
            for value in param_ndarray.flatten():
                f.write(f"{value:.8e}\n")

# Create the "32bit_float" folder in the current directory
float_folder = '32bit_float'
os.makedirs(float_folder, exist_ok=True)

# Extract encoder.0 to encoder.5
for i in range(1):
    folder_name = f'encoder_{i}_params'
    folder_path = os.path.join(float_folder, folder_name)
    os.makedirs(folder_path, exist_ok=True)
    
    for layer_name, param_tensor in model_state_dict.items():
        if layer_name.startswith(f'encoder.{i}'):
            layer_folder = os.path.join(folder_path, layer_name.replace('.', '_'))
            os.makedirs(layer_folder, exist_ok=True)
            
            extract_params(layer_name, param_tensor, layer_folder)

# Extract remaining layers
layers_to_extract = ['classifier', 'embedding', 'positional_encoding']
for layer in layers_to_extract:
    folder_name = f'{layer}_params'
    folder_path = os.path.join(float_folder, folder_name)
    os.makedirs(folder_path, exist_ok=True)
    
    for layer_name, param_tensor in model_state_dict.items():
        if layer_name.startswith(layer):
            layer_folder = os.path.join(folder_path, layer_name.replace('.', '_'))
            os.makedirs(layer_folder, exist_ok=True)
            
            extract_params(layer_name, param_tensor, layer_folder)


### 16-bit IEEE 754 binary format(Unquantized model)

In [ ]:
import os
import numpy as np
import torch
import struct

model_state_dict = model.state_dict()

def float_to_binary16(value):
    # Convert float to IEEE 754 binary16 format
    binary16 = struct.pack('>e', value)
    return ''.join(f'{b:08b}' for b in binary16)

def extract_params(layer_name, param_tensor, folder_path):
    param_ndarray = param_tensor.detach().numpy()
    
    # Save parameters as binary16 representations
    param_file = os.path.join(folder_path, 'parameters.txt')
    with open(param_file, 'w') as f:
        if len(param_ndarray.shape) == 2:
            for col in param_ndarray.T:
                for value in col:
                    binary16 = float_to_binary16(value)
                    f.write(binary16 + '\n')
        else:
            for value in param_ndarray.flatten():
                binary16 = float_to_binary16(value)
                f.write(binary16 + '\n')

# Create the "16bit_binary" folder in the current directory
binary_folder = '16bit_binary'
os.makedirs(binary_folder, exist_ok=True)

# Extract encoder.0 to encoder.5
for i in range(1):
    folder_name = f'encoder_{i}_params'
    folder_path = os.path.join(binary_folder, folder_name)
    os.makedirs(folder_path, exist_ok=True)
    
    for layer_name, param_tensor in model_state_dict.items():
        if layer_name.startswith(f'encoder.{i}'):
            layer_folder = os.path.join(folder_path, layer_name.replace('.', '_'))
            os.makedirs(layer_folder, exist_ok=True)
            
            extract_params(layer_name, param_tensor, layer_folder)

# Extract remaining layers
layers_to_extract = ['classifier', 'embedding', 'positional_encoding']
for layer in layers_to_extract:
    folder_name = f'{layer}_params'
    folder_path = os.path.join(binary_folder, folder_name)
    os.makedirs(folder_path, exist_ok=True)
    
    for layer_name, param_tensor in model_state_dict.items():
        if layer_name.startswith(layer):
            layer_folder = os.path.join(folder_path, layer_name.replace('.', '_'))
            os.makedirs(layer_folder, exist_ok=True)
            
            extract_params(layer_name, param_tensor, layer_folder)


### 32-bit 浮點數(Quantized model)

In [55]:
import os
import numpy as np
import torch

model_state_dict = model_quantized.state_dict()

ignore_keys = [
    '.scale',
    '.zero_point',
    '._packed_params.dtype'
]

def extract_packed_params(layer_name, param_tensor, folder_path):
    if '_packed_params._packed_params' in layer_name:
        layer_parts = layer_name.split('.')
        layer = model_quantized
        for part in layer_parts[:-1]:
            layer = getattr(layer, part)
        
        packed_params = getattr(layer, '_packed_params')
        
        int8_weights, int8_bias = torch.ops.quantized.linear_unpack(packed_params)
        int8_weights_nd = np.array(int8_weights.int_repr())
        int8_bias_nd = int8_bias.detach().numpy()
        
        weights_file = os.path.join(folder_path, 'quantized_weights.txt')
        with open(weights_file, 'w') as f:
            for weight in int8_weights_nd.flatten():
                f.write(f"{weight:>4}\n")
        
        bias_file = os.path.join(folder_path, 'quantized_bias.txt')
        with open(bias_file, 'w') as f:
            for bias in int8_bias_nd.flatten():  
                f.write(f"{bias:>10.8f}\n")
        
        scale_file = os.path.join(folder_path, 'quantization_scale.txt')
        with open(scale_file, 'w') as f:
            f.write(f"{int8_weights.q_scale():>10.8f}")
        
        zero_point_file = os.path.join(folder_path, 'quantization_zero_point.txt')
        with open(zero_point_file, 'w') as f:  
            f.write(f"{int8_weights.q_zero_point():>4}")
    else:
        param_ndarray = param_tensor.detach().numpy()
        
        param_file = os.path.join(folder_path, 'parameters.txt') 
        with open(param_file, 'w') as f:
            for param in param_ndarray.flatten():
                f.write(f"{param:>10.8f}\n")

float32_folder = '32float'
os.makedirs(float32_folder, exist_ok=True)

for i in range(1):
    folder_name = f'encoder_{i}_params'
    folder_path = os.path.join(float32_folder, folder_name) 
    os.makedirs(folder_path, exist_ok=True)
    
    for layer_name, param_tensor in model_state_dict.items():
        if layer_name.startswith(f'encoder.{i}') and not any(layer_name.endswith(ignore_key) for ignore_key in ignore_keys):
            layer_folder = os.path.join(folder_path, layer_name.replace('.', '_'))
            os.makedirs(layer_folder, exist_ok=True)
            
            if '_packed_params._packed_params' in layer_name:
                extract_packed_params(layer_name, param_tensor, layer_folder)
            else:
                param_ndarray = param_tensor.detach().numpy()
                param_file = os.path.join(layer_folder, 'parameters.txt')
                with open(param_file, 'w') as f:
                    for param in param_ndarray.flatten():
                        f.write(f"{param:>10.8f}\n")  

layers_to_extract = ['classifier', 'embedding', 'positional_encoding']
for layer in layers_to_extract:
    folder_name = f'{layer}_params'
    folder_path = os.path.join(float32_folder, folder_name)
    os.makedirs(folder_path, exist_ok=True)
    
    for layer_name, param_tensor in model_state_dict.items():  
        if layer_name.startswith(layer) and not any(layer_name.endswith(ignore_key) for ignore_key in ignore_keys):
            layer_folder = os.path.join(folder_path, layer_name.replace('.', '_')) 
            os.makedirs(layer_folder, exist_ok=True)
            
            extract_packed_params(layer_name, param_tensor, layer_folder)


### 16-bit IEEE 754 binary format(Quantized model)

- 整數轉成8位元的二進制
- 浮點數轉成16位元的IEEE 754二進制

In [56]:
import os
import numpy as np
import torch
import struct

model_state_dict = model_quantized.state_dict()

ignore_keys = [
    '.scale',
    '.zero_point',
    '._packed_params.dtype'
]

def float_to_bin_16bit(num):
    binary = struct.pack('>e', num)
    return ''.join('{:08b}'.format(b) for b in binary)

def int_to_bin_8bit(num):
    return '{:08b}'.format(num & 0xff)

def extract_packed_params(layer_name, param_tensor, folder_path):
    if '_packed_params._packed_params' in layer_name:
        layer_parts = layer_name.split('.')
        layer = model_quantized
        for part in layer_parts[:-1]:
            layer = getattr(layer, part)
        
        packed_params = getattr(layer, '_packed_params')
        
        int8_weights, int8_bias = torch.ops.quantized.linear_unpack(packed_params)
        int8_weights_nd = np.array(int8_weights.int_repr())
        int8_bias_nd = int8_bias.detach().numpy()
        
        weights_file = os.path.join(folder_path, 'quantized_weights.txt')
        with open(weights_file, 'w') as f:
            for weight in int8_weights_nd.flatten():
                f.write(f"{int_to_bin_8bit(weight):>8}\n")
        
        bias_file = os.path.join(folder_path, 'quantized_bias.txt') 
        with open(bias_file, 'w') as f:
            for bias in int8_bias_nd.flatten():
                f.write(f"{float_to_bin_16bit(bias):>16}\n")
        
        scale_file = os.path.join(folder_path, 'quantization_scale.txt')
        with open(scale_file, 'w') as f:
            f.write(f"{float_to_bin_16bit(int8_weights.q_scale()):>16}")
    else:
        param_ndarray = param_tensor.detach().numpy()
        
        param_file = os.path.join(folder_path, 'parameters.txt')
        with open(param_file, 'w') as f:
            for param in param_ndarray.flatten():
                if isinstance(param, np.integer):
                    f.write(f"{int_to_bin_8bit(param):>8}\n") 
                else:
                    f.write(f"{float_to_bin_16bit(param):>16}\n")

for i in range(1):
    folder_name = f'binary/encoder_{i}_params'
    os.makedirs(folder_name, exist_ok=True)
    
    for layer_name, param_tensor in model_state_dict.items():
        if layer_name.startswith(f'encoder.{i}') and not any(layer_name.endswith(ignore_key) for ignore_key in ignore_keys):
            layer_folder = os.path.join(folder_name, layer_name.replace('.', '_'))
            os.makedirs(layer_folder, exist_ok=True)
            
            if '_packed_params._packed_params' in layer_name:
                extract_packed_params(layer_name, param_tensor, layer_folder)
            else:
                param_ndarray = param_tensor.detach().numpy()
                param_file = os.path.join(layer_folder, 'parameters.txt')
                with open(param_file, 'w') as f:
                    for param in param_ndarray.flatten():
                        if isinstance(param, np.integer):
                            f.write(f"{int_to_bin_8bit(param):>8}\n")
                        else:  
                            f.write(f"{float_to_bin_16bit(param):>16}\n")

layers_to_extract = ['classifier', 'embedding', 'positional_encoding'] 
for layer in layers_to_extract:
    folder_name = f'binary/{layer}_params'
    os.makedirs(folder_name, exist_ok=True)
    
    for layer_name, param_tensor in model_state_dict.items():
        if layer_name.startswith(layer) and not any(layer_name.endswith(ignore_key) for ignore_key in ignore_keys):
            layer_folder = os.path.join(folder_name, layer_name.replace('.', '_'))
            os.makedirs(layer_folder, exist_ok=True)
            
            extract_packed_params(layer_name, param_tensor, layer_folder)
